In [ ]:
#Basic imports
import pandas as pd
import numpy as np

#General SkLearn imports
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

#Keras imports
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#Define neural network function model
def bin_net(opt, loss_f, kernel, act, n_input, nodes, n_layers, drop):
    model = Sequential()
    model.add(Dense(units = nodes, activation = act, 
                    kernel_initializer = kernel, input_dim = n_input))
    model.add(Dropout(drop))
    for n in range(n_layers):
        model.add(Dense(units = nodes, activation = act, 
                    kernel_initializer = kernel))
        model.add(Dropout(drop))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    
    model.compile(optimizer = opt, loss = loss_f, 
                  metrics = ['accuracy'])
    return model  

In [ ]:
#Read data
atr = pd.read_csv('entradas-breast.csv')
target = pd.read_csv('saidas-breast.csv')

In [ ]:
#Tune model
model = KerasClassifier(build_fn = bin_net)
param = {'batch_size': [10, 20, 30],
         'epochs': [100, 200, 500],
         'opt': ['adam', 'adamax'],
         'loss_f': ['binary_crossentropy'],
         'kernel': ['random_uniform', 'normal'],
         'act': ['relu', 'tanh'],
         'n_input': [30],
         'nodes': [8, 15, 22],
         'n_layers': [0, 1, 2, 4],
         'drop': [0.2, 0.3]}

grid = GridSearchCV(estimator = model, 
                   param_grid = param,
                   scoring = 'accuracy',
                   cv = 5,
                   n_jobs = 10,
                   verbose = 2)
grid.fit(atr, target)

In [ ]:
grid_df = pd.DataFrame(data = grid.cv_results_)
grid_df.to_csv('grid_breast.csv')

In [ ]:
#Test tuned model
param = grid.best_params_
tuned_model = KerasClassifier(build_fn = bin_net,
                              batch_size = param['batch_size'],
                              epochs = param['epochs'],
                              opt = param['opt'],
                              loss_f = param['loss_f'],
                              kernel = param['kernel'],
                              act = param['act'],
                              n_input = param['n_input'],
                              nodes = param['nodes'],
                              n_layers = param['n_layers'],
                              drop = param['drop'])
score = np.mean(cross_val_score(tuned_model, atr, target, cv = 5))
                              

In [ ]:
score

0.8858096495638573